In [1]:
interp.load.module(ammonite.ops.Path("/Users/cshao/DeepLearning.scala/jupyter-script.sc"))

In [2]:
import java.io.FileWriter
import java.util.concurrent.atomic.AtomicInteger
import scala.concurrent.duration.{SECONDS, FiniteDuration}
import scala.Long
import scala.util.{DynamicVariable, Try, Success, Failure}
import com.typesafe.config.{Config, ConfigFactory}
import akka.actor.{ Actor, ActorLogging, ActorRef, ActorSystem, Props }
import akka.event.Logging
import akka.pattern.ask
import akka.serialization.Serializer
import akka.util.Timeout
import akka.util.Timeout._
import scalaz.{@@}
import scalaz.{Functor, Applicative, Monad, Tag}
import scalaz.syntax.all._
import com.thoughtworks.tryt.covariant._
import com.thoughtworks.future._
import com.thoughtworks.continuation._
import com.thoughtworks.raii.covariant._
import com.thoughtworks.raii.asynchronous._
import com.thoughtworks.feature.Factory
// import com.thoughtworks.deeplearning.plugins._
import com.thoughtworks.each.Monadic._

import java.io.FileWriter

import java.util.concurrent.atomic.AtomicInteger

import scala.concurrent.duration.{SECONDS, FiniteDuration}

import scala.Long

import scala.util.{DynamicVariable, Try, Success, Failure}

import com.typesafe.config.{Config, ConfigFactory}

import akka.actor.{ Actor, ActorLogging, ActorRef, ActorSystem, Props }

import akka.event.Logging

import akka.pattern.ask

import akka.serialization.Serializer

import akka.util.Timeout

import akka.util.Timeout._

import scalaz.{@@}

import scalaz.{Functor, Applicative, Monad, Tag}

import scalaz.syntax.all._

import com.thoughtworks.tryt.covariant._

import com.thoughtworks.future._

import com.thoughtworks.continuation._

import com.thoughtworks.raii.covariant._

import com.thoughtworks.raii.asynchronous._

import com.thoughtworks.feature.Factory
// import com.thoughtworks.deeplearning.plugins._

import com.thoughtworks.each.Monadic._

In [ ]:
def appender[A](s: A): Unit = {
    synchronized {
        val f = new FileWriter("/tmp/log.txt", true)
        try {
            publish.md(s.toString)
            f.write(s"$s\n")
            f.flush()
        } finally {
            f.close()
        }
    }
}

In [5]:

private type RemoteCont = (Resource[UnitContinuation, Try[Actor]]) => Unit

// private case class Dispatch(cont: RemoteCont)
private case class Dispatch(contBuf: Array[Byte])
private case class Result(result: Try[Unit])

class RemoteActor extends Actor {
    override def receive = {
        case Dispatch(contBuf) =>
        val backward = sender
            val value: Try[Unit] = Success(())
        
        val system = SerializableActorSystem(context.system)
            val finalizer = new (() => Unit) with Serializable {
                def apply(): Unit = {
                    system.stop(self) 
                }
            }
        
            val release: UnitContinuation[Unit] = UnitContinuation.safeAsync[Unit] { _(finalizer()) }
            val param: Resource[UnitContinuation, Try[Unit]] = Resource(value, release)
            cont(param)
            appender("remoteCont executed without throwing")
            sender ! Result(value)
    }
}

class Remote protected (val actorSystem: ActorSystem)(implicit val timeout: Timeout) {
    appender("Invoked Remote constructor")
    
    import actorSystem.dispatcher
    
    def jump: Do[Unit] = Do.async {
        appender("Invoked remote.jump")
        (cont: RemoteCont) => {
            val newActor = actorSystem.actorOf(Props(new RemoteActor))
            (newActor ? Dispatch(cont)).onComplete {
                case Success(msg) => msg.asInstanceOf[Result].result match {
                    case Success(_) => ()
                    case Failure(err) => appender(s"remote execution error: $err")
                }
                case Failure(err) => appender(s"akka networking error: $err")
            }
        }
    }
}

object Remote {
    def apply(makeActorSystem: => ActorSystem)(implicit timeout: Timeout): Do[Remote] = {
        Do.resource {
            val actorSystem = makeActorSystem
            Resource(new Remote(actorSystem), UnitContinuation.delay {
                appender("Shutting down actorSystem")
                import actorSystem.dispatcher
                val Future(TryT(tryFinalizer)) = actorSystem.terminate.toThoughtworksFuture
                tryFinalizer.map {
                    case Success(_) => appender("actorSystem terminated")
                    case Failure(err) => appender(s"actorSystem finalizer error: $err")
                }
            }.join)
        }
    }
}


var asRef: ActorSystem = null

val asRefCont: () => ActorSystem = { () => asRef }

class SerializableActorSystem(val actorSystem: ActorSystem) extends Serializable {
    def writeReplace: Any = {
        asRef = actorSystem
        new SerializableActorSystemProxy
    }
}

class SerializableActorSystemProxy extends Serializable {
    def readResolve: Any = new SerializableActorSystem(asRefCont())
}

asRef: ActorSystem = null
asRefCont: () => ActorSystem = <function0>
defined class SerializableActorSystem
defined class SerializableActorSystemProxy

In [6]:
val ac = ActorSystem("RemoteSpecActorSystem", ConfigFactory.parseString("""
      akka {
        loglevel = "DEBUG"
        actor {
          provider = remote
          debug {
            receive = on
          }
        }
        remote {
          enabled-transports = ["akka.remote.netty.tcp"]
          netty.tcp {
            hostname = "127.0.0.1"
            port = 2552
          }
       }
      }
    """))
try {
    val sac = new SerializableActorSystem(ac)
    import java.io.{ObjectOutputStream, ByteArrayOutputStream, ObjectInputStream, ByteArrayInputStream}
    val ba = new ByteArrayOutputStream
    new ObjectOutputStream(ba).writeObject(sac)
    val sac2 = new ObjectInputStream(new ByteArrayInputStream(ba.toByteArray)).readObject.asInstanceOf[SerializableActorSystem]
    println(sac)
    println(sac2)
    println(sac.actorSystem == sac2.actorSystem)
} finally {
    ac.terminate()
}

[DEBUG] [08/28/2017 18:01:32.923] [pool-6-thread-11] [EventStream(akka://RemoteSpecActorSystem)] logger log1-Logging$DefaultLogger started
[DEBUG] [08/28/2017 18:01:32.925] [pool-6-thread-11] [EventStream(akka://RemoteSpecActorSystem)] Default Loggers started
[INFO] [08/28/2017 18:01:32.943] [pool-6-thread-11] [akka.remote.Remoting] Starting remoting
[INFO] [08/28/2017 18:01:32.955] [pool-6-thread-11] [akka.remote.Remoting] Remoting started; listening on addresses :[akka.tcp://RemoteSpecActorSystem@127.0.0.1:2552]
[INFO] [08/28/2017 18:01:32.955] [pool-6-thread-11] [akka.remote.Remoting] Remoting now listens on addresses: [akka.tcp://RemoteSpecActorSystem@127.0.0.1:2552]


: 

In [ ]:
val config: Config = ConfigFactory.parseString("""
      akka {
        loglevel = "DEBUG"
        actor {
          provider = remote
          debug {
            receive = on
          }
        }
        remote {
          enabled-transports = ["akka.remote.netty.tcp"]
          netty.tcp {
            hostname = "127.0.0.1"
            port = 2552
          }
       }
      }
    """)

def remoteSpec: Unit = {
    sys.props("sun.io.serialization.extendedDebugInfo") = "true"
    
    appender("======remoteSpec======")
    
    implicit val timeout: Timeout = FiniteDuration(10, SECONDS)
    
    val m: Do[Double] = monadic[Do] {
        val remote = Remote(ActorSystem("RemoteSpecActorSystem", config)).each
        remote.jump.each
        val x = Do.now(2.0).each
        remote.jump.each
        val y = Do.now(3.0).each
        remote.jump.each
        x * y
    }
    
    val r = m.run.blockingAwait
    appender(s"result of remoteSpec: $r")
}

In [ ]:
remoteSpec

In [ ]:
val Do[Buffer] = jump().map { actorRef: ActorRef =>
    createBuffer()
}



In [ ]:
@autoInject
case class ReleaseActor(actorRef: ActorRef) extends ReadResolveable {
    
    trait WriteReplaceable {
        
        val context = readFromThreadLocalStorage()
        
        def writeReplaceable = ReleaseActor.this
    }
    
    def readResolve = {
        new WriteReplaceable {
            
        }
    }
}

In [ ]:
trait DefaultSerializable extends Serializable {
    def readObject = {
        defaultReadObject()
    }
}

object SerializableActorSystem {
    def apply(actorSystem: ActorSystem): SerializableActorSystem
}
trait SerializableActorSystem extends DefaultSerializable {
    @transient val actorSystem: ActorSystem = null
    
    override def readObject = {
        super.readObject()
        actorSystem = readFromThreadLocal()
    }
}


In [ ]:
class A

In [ ]:
classOf[A].getName

In [ ]:
def createNewActor: Do[ActorRef]

def jumpTo(actor: ActorRef): Do[Unit]

val actor1 = createNewActor.each

Do.nested(monadic[Do] {
    val actor2 = createNewActor.each
    // on actor2
}).each
// on actor 2
createNewActor.each